# Here are some resources to help you get started
[Data Source​ Website](https://data.cms.gov/provider-data/dataset/mj5m-pzi6)

Data Dictionary: https://data.cms.gov/provider-data/sites/default/files/data_dictionaries/physician/DOC_Data_Dictionary.pdf

[Exploratory Data Analysis Tutorials](https://www.analyticsvidhya.com/blog/2021/02/introduction-to-exploratory-data-analysis-eda/)



## Some insights before cleaning dataset
* Many NPI values are repeating due to doctors and clinicians working at multiple locations.
* Certain organizatons have the same name, but have different addresses, might have to be wary while cleaning.
* Phone number column is in float data type, many of them are decimals to a power of 10

Created bulleted list to note down observations made while looking at dataset
* There is quite a significant amount of null cells in this dataset.
* NPI contains no nulls, through here we can identify how many doctors are represented.
* NPI however does include many duplicates, this is due to doctors and clinicians having multiple entries in the dataset
* There are 1674853 duplicated NPI entries, when removed it contained 1212388 entries.
* For the middle name column there is a mix of nulls, full middle names, and initals.

In [1]:
import pandas as pd
import matplotlib as plt 
import numpy as np


In [2]:
# This cell downloads the data from the cms website
! wget https://data.cms.gov/provider-data/sites/default/files/resources/69a75aa9d3dc1aed6b881725cf0ddc12_1657065926/DAC_NationalDownloadableFile.csv

--2022-07-11 18:05:03--  https://data.cms.gov/provider-data/sites/default/files/resources/69a75aa9d3dc1aed6b881725cf0ddc12_1657065926/DAC_NationalDownloadableFile.csv
Resolving data.cms.gov (data.cms.gov)... 23.46.213.187, 2a02:26f0:fe00:394::28a, 2a02:26f0:fe00:3be::28a
Connecting to data.cms.gov (data.cms.gov)|23.46.213.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 734724425 (701M) [text/csv]
Saving to: ‘DAC_NationalDownloadableFile.csv’

DAC_NationalDownloa 100%[===================>] 700.69M  27.4MB/s    in 29s     

2022-07-11 18:05:33 (24.5 MB/s) - ‘DAC_NationalDownloadableFile.csv’ saved [734724425/734724425]



In [3]:
# Running ls shows us that the DAC_NationalDownnloadableFile.csv is available in the local filesystem for this notebook
! ls

DAC_NationalDownloadableFile.csv  sample_data


In [4]:
# This line reads in the file to a pandas dataframe
df = pd.read_csv('DAC_NationalDownloadableFile.csv',encoding_errors='ignore',low_memory=False)

In [5]:
# Now we view the top of the dataframe
df.head()

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,ind_assgn,grp_assgn,adrs_id
0,1215237664,8123267630,I20130613000276,VINCOLA,SONJA,NaN,NaN,F,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,MA019692419RO61XXXRDXX300
1,1215259627,4880718386,I20100907000776,CORICA,FEDERICO,A,NaN,M,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,PR007172112PO2435XFERR500
2,1215290689,3476863275,I20151106001307,OTT,ASHLEY,MCKAIN,NaN,F,NaN,UNIVERSITY ALABAMA BIRMINGHAM - SCHOOL OF OPT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,AL365679911RO21530DRXX301
3,1215385364,9234425836,I20160908000561,QUESADA MOORE,STEPHANIE,NaN,NaN,F,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,FL324054508PA426XXSTXX400
4,1215354592,2769786136,I20160204001173,NADA,ABDELHALIM,Z,NaN,M,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,NY127011743MO427XXBROA205


In [6]:
# Here we check how big the dataframe is
df.shape

(2381036, 40)

In [7]:
print(df.columns)
#Certain columns seem to having trailing spaces in their names we should try to remove them

Index(['NPI', ' Ind_PAC_ID', ' Ind_enrl_ID', ' lst_nm', ' frst_nm', ' mid_nm',
       ' suff', ' gndr', ' Cred', ' Med_sch', ' Grd_yr', ' pri_spec',
       ' sec_spec_1', '    sec_spec_2', ' sec_spec_3', ' sec_spec_4',
       ' sec_spec_all', '    org_nm', ' org_pac_id', ' num_org_mem',
       ' adr_ln_1', ' adr_ln_2', ' ln_2_sprs', ' cty', ' st', ' zip',
       ' phn_numbr', '    hosp_afl_1', ' hosp_afl_lbn_1', ' hosp_afl_2',
       ' hosp_afl_lbn_2', ' hosp_afl_3', ' hosp_afl_lbn_3', ' hosp_afl_4',
       ' hosp_afl_lbn_4', ' hosp_afl_5', ' hosp_afl_lbn_5', ' ind_assgn',
       ' grp_assgn', ' adrs_id'],
      dtype='object')


In [8]:
df.columns = df.columns.str.strip()
print(df.columns)
#Now the column names have been trimmed and will be easier to work with

Index(['NPI', 'Ind_PAC_ID', 'Ind_enrl_ID', 'lst_nm', 'frst_nm', 'mid_nm',
       'suff', 'gndr', 'Cred', 'Med_sch', 'Grd_yr', 'pri_spec', 'sec_spec_1',
       'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all', 'org_nm',
       'org_pac_id', 'num_org_mem', 'adr_ln_1', 'adr_ln_2', 'ln_2_sprs', 'cty',
       'st', 'zip', 'phn_numbr', 'hosp_afl_1', 'hosp_afl_lbn_1', 'hosp_afl_2',
       'hosp_afl_lbn_2', 'hosp_afl_3', 'hosp_afl_lbn_3', 'hosp_afl_4',
       'hosp_afl_lbn_4', 'hosp_afl_5', 'hosp_afl_lbn_5', 'ind_assgn',
       'grp_assgn', 'adrs_id'],
      dtype='object')


In [9]:
df.info()
#using info to check on basic info about dataset such as column name and data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381036 entries, 0 to 2381035
Data columns (total 40 columns):
 #   Column          Dtype  
---  ------          -----  
 0   NPI             int64  
 1   Ind_PAC_ID      int64  
 2   Ind_enrl_ID     object 
 3   lst_nm          object 
 4   frst_nm         object 
 5   mid_nm          object 
 6   suff            object 
 7   gndr            object 
 8   Cred            object 
 9   Med_sch         object 
 10  Grd_yr          float64
 11  pri_spec        object 
 12  sec_spec_1      object 
 13  sec_spec_2      object 
 14  sec_spec_3      object 
 15  sec_spec_4      object 
 16  sec_spec_all    object 
 17  org_nm          object 
 18  org_pac_id      float64
 19  num_org_mem     float64
 20  adr_ln_1        object 
 21  adr_ln_2        object 
 22  ln_2_sprs       object 
 23  cty             object 
 24  st              object 
 25  zip             object 
 26  phn_numbr       float64
 27  hosp_afl_1      float64
 28  hosp_afl_lbn

In [10]:
#checking how many nulls are present and whether or not they will have an impact on the analysis results
df.isnull().sum()

NPI                     0
Ind_PAC_ID              0
Ind_enrl_ID             0
lst_nm                 51
frst_nm                33
mid_nm             681545
suff              2342990
gndr                    0
Cred              1772105
Med_sch                 5
Grd_yr               1474
pri_spec                0
sec_spec_1        2056062
sec_spec_2        2346270
sec_spec_3        2377416
sec_spec_4        2380414
sec_spec_all      2056062
org_nm             162614
org_pac_id         162610
num_org_mem        162610
adr_ln_1                0
adr_ln_2          1544648
ln_2_sprs         2230611
cty                     0
st                      0
zip                     0
phn_numbr          372586
hosp_afl_1         728889
hosp_afl_lbn_1     771838
hosp_afl_2        1579379
hosp_afl_lbn_2    1591858
hosp_afl_3        1966552
hosp_afl_lbn_3    1969990
hosp_afl_4        2143001
hosp_afl_lbn_4    2144748
hosp_afl_5        2229495
hosp_afl_lbn_5    2230610
ind_assgn               0
grp_assgn   

In [11]:
#Checking for how many distinct NPI's are present, since the column contains 0 nulls
len(df.NPI.unique())

1212388

In [12]:
#Because the other ID's also contain 0 nulls I will make sure they match the amount of distinct values present under NPI
len(df.Ind_PAC_ID.unique())

1212390

In [13]:
len(df.Ind_enrl_ID.unique())

1268758

In [14]:
len(df.adrs_id.unique())

349189

In [15]:
df[df.duplicated(['NPI'], keep=False)]
#Here I used  df.duplicated() to locate which rows were duplicates based off NPI

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,ind_assgn,grp_assgn,adrs_id
6,1215175666,6103004528,I20110706000450,RAI,SEEMA,NaN,NaN,F,NaN,NEW YORK MEDICAL COLLEGE,...,NORTH SHORE UNIVERSITY HOSPITAL,330195,LONG ISLAND JEWISH MEDICAL CENTER,NaN,NaN,NaN,NaN,Y,M,NY110401402NE27005AVEX300
7,1215367081,8224318365,I20161207002573,WEN,GENGYUN,NaN,NaN,M,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,CT062501693MA18XXXDRXX400
9,1215197819,5395800973,I20090213000497,FRENCH,COREY,M,NaN,M,NaN,LOGAN COLLEGE OF CHIROPRACTIC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,TN370438071CL1990XSTXX302
12,1215260591,5294056370,I20150730010948,BLANK,JEREMY,NaN,NaN,M,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,CA925914616TE27403RDXX301
14,1215263991,2668643271,I20110912000278,REINERS,COLLIN,NaN,NaN,M,NaN,NEW ENGLAND COLLEGE OF OPTOMETRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,MA010402709HO50XXXSTXX300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381030,1790253185,2769449313,I20181115003110,BASS,LONNIE,NaN,NaN,M,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,GA305123139BL35XXXRDXX300
2381032,1952926776,9638598683,I20200924003127,CHILDERS,MASON,BUCKLEY,NaN,M,NaN,UNIVERSITY ALABAMA BIRMINGHAM - SCHOOL OF OPT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,AL363601507OZ231XXSTXX400
2381033,1811202716,446428627,I20110713000454,LANGFORD,MATTHEW,STEPHEN,NaN,M,NaN,ILLINOIS COLLEGE OF OPTOMETRY AT CHICAGO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,CA921174936SA4310XAVEX302
2381034,1952394777,6901837848,I20050830000724,SNEAG,GARY,NaN,NaN,M,OD,PENNSYLVANIA COLLEGE OF OPTOMETRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,CA921174936SA4310XAVEX302


In [16]:
#Now that I know that duplicates are present I will remove them
df = df.drop_duplicates('NPI', keep='last')
df
#Now that dataframe contains 1212388 NPI values

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,ind_assgn,grp_assgn,adrs_id
0,1215237664,8123267630,I20130613000276,VINCOLA,SONJA,NaN,NaN,F,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,MA019692419RO61XXXRDXX300
1,1215259627,4880718386,I20100907000776,CORICA,FEDERICO,A,NaN,M,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,PR007172112PO2435XFERR500
2,1215290689,3476863275,I20151106001307,OTT,ASHLEY,MCKAIN,NaN,F,NaN,UNIVERSITY ALABAMA BIRMINGHAM - SCHOOL OF OPT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,AL365679911RO21530DRXX301
3,1215385364,9234425836,I20160908000561,QUESADA MOORE,STEPHANIE,NaN,NaN,F,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,FL324054508PA426XXSTXX400
4,1215354592,2769786136,I20160204001173,NADA,ABDELHALIM,Z,NaN,M,NaN,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,NY127011743MO427XXBROA205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381031,1568088722,5193140739,I20200730000687,CHILDERS,GISELLE,PACHECO,NaN,F,NaN,UNIVERSITY ALABAMA BIRMINGHAM - SCHOOL OF OPT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,AL363601507OZ231XXSTXX400
2381032,1952926776,9638598683,I20200924003127,CHILDERS,MASON,BUCKLEY,NaN,M,NaN,UNIVERSITY ALABAMA BIRMINGHAM - SCHOOL OF OPT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,AL363601507OZ231XXSTXX400
2381033,1811202716,446428627,I20110713000454,LANGFORD,MATTHEW,STEPHEN,NaN,M,NaN,ILLINOIS COLLEGE OF OPTOMETRY AT CHICAGO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,CA921174936SA4310XAVEX302
2381034,1952394777,6901837848,I20050830000724,SNEAG,GARY,NaN,NaN,M,OD,PENNSYLVANIA COLLEGE OF OPTOMETRY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,CA921174936SA4310XAVEX302


## Questions about dataset
* Is there any other way to get accurate phone numbers, changing the data type will round the number to nearest integer.
* Will removing depeating NPI's affect the information I can retrieve from this dataset?

